In [10]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction to CS203

## To sort or not to sort?

Consider the following code snippet, what the performance looks like when option is set to 1 or 0?

In [12]:
render_code("calculate_sum.c")

// calculate_sum.c:1-15 (15 lines)
#ifdef __cplusplus
extern "C" {
#endif
long long calculate_sum(int *data, unsigned int size, int threshold)
{
    long long sum=0;
    for (unsigned i = 0; i < size; ++i) {
        if (data[i] >= threshold)
                sum ++;
    }
    return sum;
}
#ifdef __cplusplus
}
#endif

In [14]:
render_code("arraySort.cpp")

// arraySort.cpp:1-37 (37 lines)
#include <algorithm>
#include <ctime>
#include <iostream>
#include <climits>

#ifdef __cplusplus
extern "C" {
#endif
long long calculate_sum(int *data, unsigned int size, int threshold);
#ifdef __cplusplus
}
#endif


int main(int argc, char **argv)
{
    // generate data
//    const unsigned arraySize = 262144;
    int arraySize = atoi(argv[1]);
    int data[arraySize];
    long long sum = 0;

    int iterations = atoi(argv[2]);
    int option = atoi(argv[3]);

    for (unsigned c = 0; c < arraySize; ++c)
        data[c] = std::rand();

    if(option)
        std::sort(data, data + arraySize);

    for (unsigned i = 0; i < iterations; ++i) {
        sum += calculate_sum(data, arraySize, std::rand());
    }
    std::cout << "sum = " << sum << std::endl;
}

In [15]:
! make clean; make

rm -f madd_A madd_B arraySort bitonic quicksort quicksort_cpu
gcc -O3 madd_A.c -o madd_A
gcc -O3 madd_B.c -o madd_B
gcc calculate_sum.c -c 
g++ -O0 arraySort.cpp calculate_sum.o -o arraySort
/usr/local/cuda/bin/nvcc -O3 bitonic.cu -o bitonic
/usr/local/cuda/bin/nvcc -O3 quicksort.cu -o quicksort
make: *** No rule to make target 'quicksort_cpu.cpp', needed by 'quicksort_cpu'.  Stop.


In [18]:
print("With sorting")
! time ./arraySort 262144 1000 1

With sorting
sum = 127161197

real	0m0.165s
user	0m0.163s
sys	0m0.002s


In [19]:
print("Without sorting")
! time ./arraySort 262144 1000 0

Without sorting
sum = 127161197

real	0m0.590s
user	0m0.587s
sys	0m0.002s


## Same complexity, similar run time?

In [20]:
compare([do_render_code("madd_A.c", lang="c++", show=["//START", "//END"]),do_render_code("madd_B.c", lang="c++", show=["//START", "//END"])])

In [21]:
! make clean; make madd_A; make madd_B
print("i outside")
! time ./madd_A 16384
print("j outside")
! time ./madd_B 16384

rm -f madd_A madd_B arraySort bitonic quicksort quicksort_cpu
gcc -O3 madd_A.c -o madd_A
gcc -O3 madd_B.c -o madd_B
i outside
88

real	0m6.144s
user	0m5.445s
sys	0m0.698s
j outside
88

real	0m9.257s
user	0m8.586s
sys	0m0.669s


## Are bitwise operations faster?

In [22]:
compare([do_render_code("swap.c", lang="c++", show="regswap"),do_render_code("swap.c", lang="c++", show="xorswap")])

In [24]:
! rm -f ./swap; gcc -o swap swap.c
print("naive swap")
! ./swap 524288 0
print("xor swap")
! ./swap 524288 1

naive swap
data[rand()] = 189276545
sorted 0.046348 seconds
xor swap
data[rand()] = 189276545
sorted 0.050205 seconds


## Algorithm on GPUs?
Consider quicksort and bitsort, which performs better on GPUs?

### Quick sort

In [24]:
render_code("quicksort.cu", show="quickSortIterative")

// quicksort.cu:62-99 (38 lines)
void quickSortIterative (float arr[], int l, int h)
{
    int lstack[ h - l + 1 ], hstack[ h - l + 1];
    float *d_d;
    int top = -1, *d_l, *d_h;
 
    lstack[ ++top ] = l;
    hstack[ top ] = h;

    cudaMalloc(&d_d, (h-l+1)*sizeof(float));
    cudaMemcpy(d_d, arr,(h-l+1)*sizeof(float),cudaMemcpyHostToDevice);

    cudaMalloc(&d_l, (h-l+1)*sizeof(int));
    cudaMemcpy(d_l, lstack,(h-l+1)*sizeof(int),cudaMemcpyHostToDevice);

    cudaMalloc(&d_h, (h-l+1)*sizeof(int));
    cudaMemcpy(d_h, hstack,(h-l+1)*sizeof(int),cudaMemcpyHostToDevice);
    int n_t = 1;
    int n_b = 1;
    int n_i = 1; 
    while ( n_i > 0 )
    {
        partition<<<n_b,n_t>>>( d_d, d_l, d_h, n_i);
        int answer;
        cudaMemcpyFromSymbol(&answer, d_size, sizeof(int), 0, cudaMemcpyDeviceToHost); 
        if (answer < 1024)
          {
            n_t = answer;
          }
        else
          {
            n_t = 1024;
            n_b = answer/n_t + (answer%n_t==0?0:1);
          }
        n_i = answer;
        cudaMemcpy(arr, d_d,(h-l+1)*sizeof(int),cudaMemcpyDeviceToHost);
    }
}

In [25]:
render_code("quicksort.cu", show="partition")

// quicksort.cu:21-60 (40 lines)
__global__ void partition (float *arr, int *arr_l, int *arr_h, int n)
{
    int z = blockIdx.x*blockDim.x+threadIdx.x;
    d_size = 0;
    __syncthreads();
    if (z<n)
      {
        int h = arr_h[z];
        int l = arr_l[z];
        int x = arr[h];
        int i = (l - 1);
        int temp;
        for (int j = l; j <= h- 1; j++)
          {
            if (arr[j] <= x)
              {
                i++;
                temp = arr[i];
                arr[i] = arr[j];
                arr[j] = temp;
              }
          }
        temp = arr[i+1];
        arr[i+1] = arr[h];
        arr[h] = temp;
        int p = (i + 1);
        if (p-1 > l)
          {
            int ind = atomicAdd(&d_size, 1);
            arr_l[ind] = l;
            arr_h[ind] = p-1;  
          }
        if ( p+1 < h )
          {
            int ind = atomicAdd(&d_size, 1);
            arr_l[ind] = p+1;
            arr_h[ind] = h; 
          }
      }
}

### Bitonic sort

In [26]:
render_code("bitonic.cu", show="bitonic_sort")

// bitonic.cu:81-102 (22 lines)
void bitonic_sort(float *values)
{
  float *dev_values;
  size_t size = NUM_VALS * sizeof(float);

  cudaMalloc((void**) &dev_values, size);
  cudaMemcpy(dev_values, values, size, cudaMemcpyHostToDevice);

  dim3 blocks(BLOCKS,1);    /* Number of blocks   */
  dim3 threads(THREADS,1);  /* Number of threads  */

  int j, k;
  /* Major step */
  for (k = 2; k <= NUM_VALS; k <<= 1) {
    /* Minor step */
    for (j=k>>1; j>0; j=j>>1) {
      bitonic_sort_step<<<blocks, threads>>>(dev_values, j, k);
    }
  }
  cudaMemcpy(values, dev_values, size, cudaMemcpyDeviceToHost);
  cudaFree(dev_values);
}

In [27]:
render_code("bitonic.cu", show="bitonic_sort_step")

// bitonic.cu:49-76 (28 lines)
__global__ void bitonic_sort_step(float *dev_values, int j, int k)
{
  unsigned int i, ixj; /* Sorting partners: i and ixj */
  i = threadIdx.x + blockDim.x * blockIdx.x;
  ixj = i^j;

  /* The threads with the lowest ids sort the array. */
  if ((ixj)>i) {
    if ((i&k)==0) {
      /* Sort ascending */
      if (dev_values[i]>dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
    if ((i&k)!=0) {
      /* Sort descending */
      if (dev_values[i]<dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
  }
}

In [25]:
! make bitonic
! ./bitonic

/usr/local/cuda/bin/nvcc -O3 bitonic.cu -o bitonic
Sorting 32768 numbers
Elapsed time: 0.288s


In [26]:
! make quicksort
!./quicksort

/usr/local/cuda/bin/nvcc -O3 quicksort.cu -o quicksort
Sorting 32768 numbers
Elapsed time: 33.844s


In [27]:
! nvidia-smi -a


==============NVSMI LOG==============

Timestamp                                 : Thu Sep 26 10:27:51 2024
Driver Version                            : 555.42.06
CUDA Version                              : 12.5

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Product Name                          : NVIDIA GeForce RTX 4070 Ti SUPER
    Product Brand                         : GeForce
    Product Architecture                  : Ada Lovelace
    Display Mode                          : Disabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
  